In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import math
import time
import random

In [4]:
os.chdir("E:/AI/assignment_2_online")

In [5]:
romania=pd.read_pickle("romania_graph.pickle")
atlanta=pd.read_pickle("atlanta_osm.pickle")

In [6]:
print(nx.info(atlanta))

Name: 
Type: Graph
Number of nodes: 252791
Number of edges: 271652
Average degree:   2.1492


In [7]:
def ucs(graph,start,goal):
    start_time=time.time()
    frontier={start:(0,"start")}
    explored={} 
    while True:
        if frontier=={}:
            print("Failed to find a path.")
            eclipse_time=time.time()-start_time
            return "Falied to find the path.",0,eclipse_time
            break
        shortest_dist=float('inf')
        for key, value in frontier.items():           
            if value[0]<shortest_dist:
                shortest_dist=value[0]
                to_be_explored=key
        explored[to_be_explored]=frontier[to_be_explored]
        for key,value in graph[to_be_explored].items():
            if key in explored.keys():
                continue
            if key in frontier.keys():
                new_distance=frontier[to_be_explored][0]+value['weight']
                if new_distance<frontier[key][0]:
                    frontier[key]=(new_distance,to_be_explored)
            else:
                distance=frontier[to_be_explored][0]+value['weight']
                frontier[key]=(distance,to_be_explored)
        frontier.pop(to_be_explored)    
        if to_be_explored==goal:
            invers_path=[goal]
            while True:
                next=invers_path[-1]
                if next==start:
                    path=invers_path[::-1]
                    eclipse_time=time.time()-start_time
                    return path,explored[to_be_explored][0],eclipse_time
                else:
                    invers_path.append(explored[next][1])

In [8]:
def heuristic(graph,a,b):
    return math.sqrt((graph.node[a]['pos'][0]-graph.node[b]['pos'][0])**2+(graph.node[a]['pos'][1]-graph.node[b]['pos'][1])**2)

In [9]:
def Astar(graph,start,goal):
    start_time=time.time()
    frontier={start: (0, heuristic(graph,start,goal),"start")}
    explored={}
    while True:
        if frontier=={}:
            print("Failed to find a path.")
            eclipse_time=time.time()-start_time
            return "Falied to find the path.",0,eclipse_time
            break
        shortest=float('inf')
        for key,value in frontier.items():
            if value[0]+value[1]<shortest:
                shortest=value[0]+value[1]
                to_be_explored=key
        explored[to_be_explored]=(frontier[to_be_explored][0],frontier[to_be_explored][2])
        if to_be_explored==goal:
            invers_path=[goal]
            while True:
                next=invers_path[-1]
                if next==start:
                    path=invers_path[::-1]
                    return path,explored[to_be_explored][0],time.time()-start_time
                else:
                    invers_path.append(explored[next][1])
        for key,value in graph[to_be_explored].items():
            if key in explored.keys():
                continue
            elif key in frontier.keys():
                distance=frontier[to_be_explored][0]+value['weight']
                if distance<frontier[key][0]:
                    frontier[key]=(distance,heuristic(graph,key,goal),to_be_explored)
            else:
                distance=frontier[to_be_explored][0]+value['weight']
                frontier[key]=(distance,heuristic(graph,key,goal),to_be_explored)
        frontier.pop(to_be_explored)

In [10]:
def getpath(middlepoint, explored1,explored2,start,goal):
    invers_halfpath=[middlepoint]
    while True:
        next=invers_halfpath[-1]
        if next==start:
            break
        else:
            invers_halfpath.append(explored1[next][1])
    halfpath=[middlepoint]
    while True:
        next=halfpath[-1]
        if next==goal:
            break
        else:
            halfpath.append(explored2[next][1])
    path=invers_halfpath[::-1]+halfpath[1:]
    return path

In [11]:
def bidirection_ucs(graph,start,goal):
    start_time=time.time()
    frontier1={start:(0,"start")}
    frontier2={goal:(0,"goal")}
    explored1={}
    explored2={}
    while True:
        if frontier1=={} or frontier2=={}:
            print("Failed to find a path.")
            eclipse_time=time.time()-start_time
            return "Falied to find the path.",0,eclipse_time
            break
        shortest_dist1=float('inf')
        for key, value in frontier1.items():           
            if value[0]<shortest_dist1:
                shortest_dist1=value[0]
                to_be_explored1=key
        explored1[to_be_explored1]=frontier1[to_be_explored1]
        shortest_dist2=float('inf')
        for key,value in frontier2.items():
            if value[0]<shortest_dist2:
                shortest_dist2=value[0]
                to_be_explored2=key
        explored2[to_be_explored2]=frontier2[to_be_explored2]   
        for key,value in graph[to_be_explored1].items():
            if key in explored1.keys():
                continue
            if key in frontier1.keys():
                new_distance=frontier1[to_be_explored1][0]+value['weight']
                if new_distance<frontier1[key][0]:
                    frontier1[key]=(new_distance,to_be_explored1)
            else:
                distance=frontier1[to_be_explored1][0]+value['weight']
                frontier1[key]=(distance,to_be_explored1)
        for key,value in graph[to_be_explored2].items():
            if key in explored2.keys():
                continue
            if key in frontier2.keys():
                new_distance=frontier2[to_be_explored2][0]+value['weight']
                if new_distance<frontier2[key][0]:
                    frontier2[key]=(new_distance,to_be_explored2)
            else:
                distance=frontier2[to_be_explored2][0]+value['weight']
                frontier2[key]=(distance,to_be_explored2)
        frontier1.pop(to_be_explored1)
        frontier2.pop(to_be_explored2)
        if to_be_explored1 in explored2.keys():
            path=getpath(to_be_explored1, explored1,explored2,start,goal)
            distance=explored1[to_be_explored1][0]+explored2[to_be_explored1][0]
            time_eclipse=time.time()-start_time
            return path,distance,time_eclipse
        elif to_be_explored2 in explored1.keys():
            path=getpath(to_be_explored2,explored1,explored2,start,goal)
            distance=explored1[to_be_explored2][0]+explored2[to_be_explored2][0]
            time_eclipse=time.time()-start_time
            return path,distance,time_eclipse            

In [14]:
start='a'
goal='b'
graph=romania
episode=100

In [ ]:
path1,distance1,eclipse_time1=ucs(graph,start,goal)
print("Distance between start and goal is %f."%(distance1))
print("Time used for searching is %f."%(eclipse_time1))

In [ ]:
path2,distance2,eclipse_time2=Astar(graph,start,goal)
print("Distance between start and goal is %f."%(distance2))
print("Time used for searching is %f."%(eclipse_time2))

In [15]:
path3,distance3,eclipse_time3=bidirection_ucs(graph,start,goal)
print("Distance between start and goal is %f."%(distance3))
print("Time used for searching is %f."%(eclipse_time3))

Distance between start and goal is 418.000000.
Time used for searching is 0.000000.


In [16]:
different_path=0
time1=0
time2=0
time3=0
for i in range(episode):
    start,goal=random.sample(atlanta.nodes(),2)
    path1,distance1,eclipse_time1=ucs(atlanta,start,goal)
    path2,distance2,eclipse_time2=Astar(atlanta,start,goal)
    path3,distance3,eclipse_time3=bidirection_ucs(atlanta,start,goal)
    if distance1-distance2>distance1*0.00001 or distance1-distance3>distance1*0.00001:
        different_path+=1
    time1+=eclipse_time1
    time2+=eclipse_time2
    time3+=eclipse_time3
print("Average time needed for usc, Astar and bi-usc is %f,%f,%f respectively."%(time1/100,time2/100,time3/100))  
    

Failed to find a path.
Failed to find a path.
Failed to find a path.
Average time needed for usc, Astar and bi-usc is 6.342769,10.113235,4.706228 respectively.


In [17]:
print("Average time needed for Uniform Cost Search is %fs."%(time1/100))
print("Average time needed for A Star is %fs."%(time2/100))
print("Average time needed for Bi-direction Uniform Cost Search is %fs."%(time3/100))

Average time needed for Uniform Cost Search is 6.342769s.
Average time needed for A Star is 10.113235s.
Average time needed for Bi-direction Uniform Cost Search is 4.706228s.


Summary: In my experiment, bi-direction ucs is the fastest, because on average it searches less nodes, especially if the start node is far away from the goal. 
A Star needs more time because heuristic distance needs to be calculated. 